In [1]:
# Transfer Learning(전이학습)
# Transfer Learning(전이학습)이란 아주 큰 데이터셋, 즉 21,841 부류에 대해서 총 1419만7122장의 이미지로 구성되어 있는
# ImageNet 데이터를 사용해서 학습된 모델의 가중치를 가져와서, 우리가 해결하려는 문제에 맞게 보정해서 사용하는 것을 의미함
# 이때 큰 데이터셋을 사용해서 훈련된 모델을 사전학습모델(pre-trained model)이라고 함
# ImageNet 데이터의 이미지 크기는 평균적으로 469x387 이며, 이러한 2만개 이상의 부류 가운데 1000 부류만 뽑아서 데이터를 구성하고
# 정확도를 높이는 대회가 바로 ImageNet Challenge 이다

# 전이학습 필요성 
# ImageNet(풍부한 데이터) -> 사전학습모델(pre-trained model) -> 전이학습(Transfer Learning) -> 새롭게 학습되는 모델(일반적으로 파인튜닝으로 사용됨) <- 부족한 사용자 데이터

# 사전 학습 모델 구조(pre-trained model)
# 입력 -> 사전 학습된 특징 추출기(pre-trained feature extractor) -> 사전 학습된 분류기(pre-trained classifier) -> 출력

# 사전 학습된 특징 추출기(pre-trained feature extractor)
# - 특징 추출기(feature extractor)는 컨볼루션층과 풀링층의 조합으로 구성되어 있으며, ImageNet 데이터에 대해 이미 학습되어 있음
# - conv->conv->pooling->, 특징 추출기는 출력 데이터를 bottleneck 또는 feature vector 등으로 지칭함

# 사전 학습된 분류기(pre-trained classifier)
# - 분류기(classifier)는 완전 연결층으로 구성되며 추출된 특징을 입력으로 받아 최종적으로 주어진 이미지에 대한 클래스(정답)을 카테고리 형태로 분류하는 역할 수행
# - Linear->Linear->Linear(Softmax)->, 오버피팅을 줄이기 위해 출력층 이전의 Linear layer 사이에는 Dropout, BatchNormalization layer 등을 추가 할 수 있음

# 파인 튜닝(Fine Tuning)
# - 사전 학습 모델의 가중치를 미세하게 조정하는 기법이며, 새롭게 분류하려는 데이터의 종류와 전체 개수를 미리 분석한 후에, 
#   그것을 바탕으로 사전 학습 모델 가중치 일부만을 재학습 시키거나 또는 모든 가중치를 처음부터 다시 학습시킬 수 있음

In [7]:
# pre-trained model(사전학습모델) 다운로드
import torch
from torch import nn
from torchvision import models # 다양한 사전 학습 모델을 포함한 모듈

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('pytorch version : ', torch.__version__, ', device : ', DEVICE)

# 사전 학습 모델 vgg16(), 사전 학습된 가중치 weights=models.VGG16_Weights.DEFAULT
pretrained_model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)

print(pretrained_model)

pytorch version :  2.2.2 , device :  cpu
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kerne

In [ ]:
# Transfer Learning Model
class TransferLearningModel(nn.Module):
    def __init__(self, pretrained_model, feature_extractor):
        super().__init__()

        if(feature_extractor):
            for param in pretrained_model.parameters():
                param.requires_grad = False # 특정 파라미터의 기울기 계산을 중단시키는 설정, 가중치/바이어스 학습과정에서 업데이트 되지 않음
        
        # 학습데이터에 맞게 새로운 분류기를 만들어 준 후에, 기존 사전학습모델 classifier 부분을 새로운 classifier 로 반드시 바꾸어야 함
        pretrained_model.classifier = nn.Sequential(
            nn.Linear(pretrained_model.classifier[0].in_features, 128),
            nn.Linear(128, 2)
        )
        self.model = pretrained_model
    
    def forward(self, x):
        x = self.model(x)
        return x

In [8]:
feature_extractor = True # True : feature_extractor, False : Fine Tuning

# model 객체 생성
model = TransferLearningModel(pretrained_model, feature_extractor).to(DEVICE)
# loss function
loss_function = nn.CrossEntropyLoss()
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-6)